# filter bubble effect (working title)

### #pre import needed packages

* data/us_senetors_040722.csv
* data/us_representatives_040722.csv


In [1]:
#imports

#!pip install vaderSentiment
#!pip install twarc

from datetime import date, datetime, timezone
import asyncio
import json
import pandas as pd
pd.options.display.max_colwidth = 400
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime, timezone, timedelta
import csv

from twarc import Twarc2 #twitter

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #sentiment vader
sentimentAnalyser = SentimentIntensityAnalyzer() #sentiment vader

### #1 import of members of congress (senators, representatives) and keywords on 'abortion'

* data/us_senators_040722.csv
* data/us_representatives_040722.csv


In [2]:
#file imports (all tweets from filename.csv)
senators = []
representatives = []
keywords=[]

#read
senators=pd.read_csv('data/us_senators_040722.csv', header='infer', sep=";", dtype=str)
representatives=pd.read_csv('data/us_representatives_040722.csv', header='infer', sep=";", dtype=str)

#add twitter handle (extraction from link)
senators['twitter'] = senators['Link'].str.replace('https://twitter.com/','')
representatives['twitter'] = representatives['Link'].str.replace('https://twitter.com/','')

#keyword to list
with open('data/keywords.csv', 'r') as read_keywords:
    next(read_keywords)
    lines = read_keywords.readlines()
    for line in lines:
            line = line.replace("\n", "")
            keywords.append(line)

#prints---------
print("Senators: ", len(senators))
print("Representatives: ",len(representatives))
print()
#print(keywords)
#display(senators.head(5))
#display(representatives.head(5))

Senators:  100
Representatives:  436



/var/folders/qh/_dtmnfpx1552ksv4hpb8ly3h0000gn/T/ipykernel_4408/1323361081.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  senators['twitter'] = senators['Link'].str.replace('https://twitter.com/','')
/var/folders/qh/_dtmnfpx1552ksv4hpb8ly3h0000gn/T/ipykernel_4408/1323361081.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  representatives['twitter'] = representatives['Link'].str.replace('https://twitter.com/','')


### #2 lets focus on senators and process a twitter query

* https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/5-how-to-write-search-queries.md

#### it should include:

* (a)  English as language ("lang:en")
* (b)  members of congress ("from:HANDLE OR from:HANDLE OR ...")
* (c)  keywords for abortion (list provided by Jana)


!queries for senators are devided in 6 queries due to character size of query!

In [3]:
#old

#todo - what query do we want?

#a
#query=str("lang:en ")

#b
#for index, row in senators.iterrows():
#    query=query+str("from:")+str(row['twitter'])+str(" OR ")
#c
#for entry in keywords:
#    query=query+str(entry)+str(" OR ")

#prints---------
#print("'query' done!")
#print()
#print("query=",query, sep="")

In [4]:
#query split

#query language and keywords (323 characters)
query1=str("lang:en (abortion OR pregnancies OR pregnancy OR pregnant OR unborn OR embryonic OR pro choice OR pro life OR prochoice OR prolife OR roevwade OR roe v wade OR right to life OR righttolife OR infanticide OR parenthood OR abortions OR embryo OR contraception OR ultrasound OR planned parenthood OR reproductive OR ultrasounds)")

#senators
query2a=str(" AND (from:SenatorBaldwin OR from:SenJohnBarrasso OR from:SenatorBennet OR from:MarshaBlackburn OR from:SenBlumenthal OR from:RoyBlunt OR from:CoryBooker OR from:JohnBoozman OR from:SenatorBraun OR from:SenSherrodBrown OR from:SenatorBurr OR from:SenatorCantwell OR from:SenCapito OR from:SenatorCardin OR from:SenatorCarper OR from:SenBobCasey OR from:SenBillCassidy OR from:SenatorCollins)") 

query2b=str(" AND (from:ChrisCoons OR from:JohnCornyn OR from:SenCortezMasto OR from:SenTomCotton OR from:SenKevinCramer OR from:MikeCrapo OR from:SenTedCruz OR from:SteveDaines OR from:SenDuckworth OR from:SenatorDurbin OR from:SenJoniErnst OR from:SenFeinstein OR from:SenatorFischer OR from:SenGillibrand OR from:LindseyGrahamSC OR from:ChuckGrassley OR from:SenatorHagerty OR from:SenatorHassan)")  

query2c=str(" AND (from:HawleyMO OR from:MartinHeinrich OR from:SenatorHick OR from:maziehirono OR from:SenJohnHoeven OR from:SenHydeSmith OR from:JimInhofe OR from:SenRonJohnson OR from:timkaine OR from:SenMarkKelly OR from:SenJohnKennedy OR from:SenAngusKing OR from:SenAmyKlobuchar OR from:SenatorLankford OR from:SenatorLeahy OR from:SenMikeLee OR from:SenatorLujan OR from:SenLummis OR from:Sen_JoeManchin OR from:SenMarkey OR from:SenatorMarshall)") 

query2d=str(" AND (from:senatemajldr OR from:SenatorMenendez OR from:SenJeffMerkley OR from:JerryMoran OR from:lisamurkowski OR from:ChrisMurphyCT OR from:PattyMurray OR from:ossoff OR from:SenAlexPadilla OR from:RandPaul OR from:SenGaryPeters OR from:senrobportman OR from:SenJackReed OR from:SenatorRisch OR from:SenatorRomney OR from:SenJackyRosen OR from:SenatorRounds OR from:marcorubio OR from:SenSanders)")

query2e=str(" AND (from:SenSasse OR from:SenBrianSchatz OR from:SenSchumer OR from:SenRickScott OR from:SenatorTimScott OR from:SenatorShaheen OR from:SenShelby OR from:SenatorSinema OR from:SenTinaSmith OR from:SenStabenow OR from:SenDanSullivan OR from:SenatorTester OR from:SenJohnThune OR from:SenThomTillis OR from:SenToomey OR from:SenTuberville OR from:ChrisVanHollen OR from:MarkWarner OR from:SenatorWarnock OR from:SenWarren OR from:SenWhitehouse OR from:SenatorWicker OR from:RonWyden OR from:SenToddYoung)")


### #3 lets focus on the senators-df only and download the tweets

* define timespan (start, end) for download
* download tweets using the computed query to dataframe 'tweets'
* converting columns
    * 'author_id','id','conversation_id' to string
    * 'created_at' to date and setting it as index

In [5]:
#TIMESPAN:
#Today(last midnight) minus 6 days
today=datetime.now()
timespan_days=6
timespan = today - timedelta(days=timespan_days) #minus 6 days
#calculated
start = datetime(timespan.year, timespan.month, timespan.day, 0, 0, 0, 0, tzinfo=timezone.utc)
end = datetime(today.year, today.month, today.day, 0, 0, 0, 0, tzinfo=timezone.utc)

#manual set of start- and start- and enddates for twitter-elevated access
start = datetime(2022, 7, 9, 0, 0, 0, 0, tzinfo=timezone.utc) #we should see a peak
end = datetime(2022, 7, 23, 0, 0, 0, 0, tzinfo=timezone.utc) 


#TWITTER
#bearer_token
bearer_token_academic_summerschool = "AAAAAAAAAAAAAAAAAAAAAHCpewEAAAAAp0ENemYtS4sb4uUoKufBXd4zoo0%3DOFrhxhAmsi15Gq7WHFUg3e0Dlcd9yyVBKmH7Zg98keVfupTjJm"

In [ ]:
#QueryA
queryA=query1+query2a
query=query1

bearer_token = bearer_token_academic_summerschool
twarc_client = Twarc2(bearer_token=bearer_token)
results = [] #df for "new" tweets
for tweet in twarc_client.search_all(
        query,
        start_time=start,
        end_time=end):
    results.append(tweet)
    
tweets = []
tweets = pd.json_normalize(results, record_path=['data']) #creating dataframe

#column conversion (datatypes)
tweets = tweets.astype({'author_id': 'str', 'id': 'str','conversation_id': 'str'})
tweets['created_at']= pd.to_datetime(tweets['created_at'])
#setting index to date
tweets = tweets.set_index("created_at")

#prints
#print ("tweets_dates:")
#print("   min:", tweets.index.min())
#print("   max:", tweets.index.max())
print()
print("Number of downloaded tweets: ", len(tweets))
print()
display(tweets.head(2))
tweets2a=tweets
print("No. of tweets in queryA: ", len(tweets2a))
display(tweets2a.head(2))

rate limit exceeded: sleeping 85.33943700790405 secs
Hit the 1 request/second rate limit, sleeping for 10 seconds. This shouldn't happen with normal usage of twarc, and may indicate multiple clients interacting with the Twitter API at the same time.
rate limit exceeded: sleeping 117.91361999511719 secs
Hit the 1 request/second rate limit, sleeping for 10 seconds. This shouldn't happen with normal usage of twarc, and may indicate multiple clients interacting with the Twitter API at the same time.
Hit the 1 request/second rate limit, sleeping for 10 seconds. This shouldn't happen with normal usage of twarc, and may indicate multiple clients interacting with the Twitter API at the same time.


In [ ]:
#QueryB
queryB=query1+query2b
query=queryB

bearer_token = bearer_token_academic_summerschool
twarc_client = Twarc2(bearer_token=bearer_token)
results = [] #df for "new" tweets
for tweet in twarc_client.search_all(
        query,
        start_time=start,
        end_time=end):
    results.append(tweet)
    
tweets = []
tweets = pd.json_normalize(results, record_path=['data']) #creating dataframe

#column conversion (datatypes)
tweets = tweets.astype({'author_id': 'str', 'id': 'str','conversation_id': 'str'})
tweets['created_at']= pd.to_datetime(tweets['created_at'])
#setting index to date
tweets = tweets.set_index("created_at")

#prints
#print ("tweets_dates:")
#print("   min:", tweets.index.min())
#print("   max:", tweets.index.max())
#print()
#print("Number of downloaded tweets: ", len(tweets))
#print()
#display(tweets.head(2))
tweets2b=tweets
print("No. of tweets in queryB: ", len(tweets2b))
display(tweets2b.head(2))

In [ ]:
#QueryC
queryC=query1+query2c
query=queryC

bearer_token = bearer_token_academic_summerschool
twarc_client = Twarc2(bearer_token=bearer_token)
results = [] #df for "new" tweets
for tweet in twarc_client.search_all(
        query,
        start_time=start,
        end_time=end):
    results.append(tweet)
    
tweets = []
tweets = pd.json_normalize(results, record_path=['data']) #creating dataframe

#column conversion (datatypes)
tweets = tweets.astype({'author_id': 'str', 'id': 'str','conversation_id': 'str'})
tweets['created_at']= pd.to_datetime(tweets['created_at'])
#setting index to date
tweets = tweets.set_index("created_at")

#prints
#print ("tweets_dates:")
#print("   min:", tweets.index.min())
#print("   max:", tweets.index.max())
#print()
#print("Number of downloaded tweets: ", len(tweets))
#print()
#display(tweets.head(2))
tweets2c=tweets
print("No. of tweets in queryC: ", len(tweets2c))
display(tweets2c.head(2))

In [ ]:
#QueryD
queryD=query1+query2d
query=queryD

bearer_token = bearer_token_academic_summerschool
twarc_client = Twarc2(bearer_token=bearer_token)
results = [] #df for "new" tweets
for tweet in twarc_client.search_all(
        query,
        start_time=start,
        end_time=end):
    results.append(tweet)
    
tweets = []
tweets = pd.json_normalize(results, record_path=['data']) #creating dataframe

#column conversion (datatypes)
tweets = tweets.astype({'author_id': 'str', 'id': 'str','conversation_id': 'str'})
tweets['created_at']= pd.to_datetime(tweets['created_at'])
#setting index to date
tweets = tweets.set_index("created_at")

#prints
#print ("tweets_dates:")
#print("   min:", tweets.index.min())
#print("   max:", tweets.index.max())
#print()
#print("Number of downloaded tweets: ", len(tweets))
#print()
#display(tweets.head(2))
tweets2d=tweets
print("No. of tweets in queryD: ", len(tweets2d))
display(tweets2d.head(2))

In [ ]:
#QueryE
queryE=query1+query2e
query=queryE

bearer_token = bearer_token_academic_summerschool
twarc_client = Twarc2(bearer_token=bearer_token)
results = [] #df for "new" tweets
for tweet in twarc_client.search_all(
        query,
        start_time=start,
        end_time=end):
    results.append(tweet)
    
tweets = []
tweets = pd.json_normalize(results, record_path=['data']) #creating dataframe

#column conversion (datatypes)
tweets = tweets.astype({'author_id': 'str', 'id': 'str','conversation_id': 'str'})
tweets['created_at']= pd.to_datetime(tweets['created_at'])
#setting index to date
tweets = tweets.set_index("created_at")

#prints
#print ("tweets_dates:")
#print("   min:", tweets.index.min())
#print("   max:", tweets.index.max())
#print()
#print("Number of downloaded tweets: ", len(tweets))
#print()
#display(tweets.head(2))
tweets2e=tweets
print("No. of tweets in queryE: ", len(tweets2e))
display(tweets2e.head(2))

### #4 Putting Sentiment-Analysis over the tweets

using Vader sentiment analysis

* which can be seen in the columns '' and ''

In [ ]:
#function for calculating vader-sentiment for whole phrase (column"text")
def vader_sentiment_score(text):
    scores = sentimentAnalyser.polarity_scores(text) # Run VADER on the text
    compound_score = scores['compound'] # Extract the compound score, whole phase
    return compound_score # Return compound score

def vader_sentiment_phrase(vader_sentiment_score):
    #scores = sentimentAnalyser.polarity_scores(text) # Run VADER on the text
    #compound_score = scores['compound'] # Extract the compound score, whole phase
    if vader_sentiment_score >= 0.05 :
        compound_phrase=str("positive")
    elif vader_sentiment_score <= - 0.05 :
        compound_phrase=str("negative")
    else :
        compound_phrase=str("neutral")
    return compound_phrase # Return compound phrase

In [ ]:
#test, if the analysis works
vader_sentiment_score('I like the Marvel movies') 

In [ ]:
#creating column 
tweets_sentiment=[]
tweets_sentiment=tweets
tweets_sentiment['vader_sentiment_score'] = tweets_sentiment['text'].apply(vader_sentiment_score)
tweets_sentiment['vader_sentiment_phrase'] = tweets_sentiment['vader_sentiment_score'].apply(vader_sentiment_phrase)

#prints---------
print()
print("Number of tweets with sentiments: ", len(tweets_sentiment))
print()
display(tweets_sentiment.head(4))

### #5 Saving to File

to 'data/downloaded_tweets.csv"

In [ ]:
date=str(today.strftime("%Y"))+str("-")+str(today.strftime("%m"))+str("-")+str(today.strftime("%d"))
filename=str("data/downloaded_tweets")+"_"+date+str("")+str(".csv")
tweets_sentiment.to_csv(filename, index=True)

### #6 Visualization

of sentiments over time only

In [ ]:
#sorting by date
tweets_sentiment.sort_values(by=['created_at'], inplace=True, ascending=True)
#display(tweets_sentiment)

tweets_sentiment['vader_sentiment_score'].plot(
    title=f"query: '{query}' - Tweet's Vader Sentiment Score");

